In [ ]:
# %load run_motion.py
import os
import numpy as np
import cv2
from scipy.interpolate import RectBivariateSpline
from skimage.filters import apply_hysteresis_threshold
import time

def lucas_kanade_affine(img1, img2, p, Gx, Gy):
    ### START CODE HERE ###
    # [Caution] From now on, you can only use numpy and 
    # RectBivariateSpline. Never use OpenCV.
    temp = img1.copy()
    img_norm  = img2.copy()/255.0
    Gx_norm = Gx.copy()/255.0
    Gy_norm = Gy.copy()/255.0
    height, width = temp.shape
    
    x = np.arange(0,width,1)
    y = np.arange(0,height,1)
    img_spline = RectBivariateSpline(y, x, img_norm)
    Gx_spline  = RectBivariateSpline(y, x, Gx_norm)
    Gy_spline  = RectBivariateSpline(y, x, Gy_norm)
    
    H = np.zeros((6,6))
    error_sum = np.zeros((6,1))
    
    for y_ in y:
        for x_ in x:
            wx = (1 + p[0,0])*x_ + p[2,0]*y_ + p[4,0]
            wy = p[1,0]*x_ + (1 + p[3,0])*y_ + p[5,0]
            if (wx >= 0 or wx <= width-1 or wy >= 0 or wy <= height-1):
                gx = float(Gx_spline(wy,wx)*255)
                gy = float(Gy_spline(wy,wx)*255)
                error = temp[y_,x_] - float(img_spline.ev(wy,wx)*255)
                grad = np.array([[gx*x_],[gy*x_],[gx*y_],[gy*y_],[gx],[gy]])
                error_sum = error_sum + grad*error
                H = H + grad@grad.T

    inv_H = np.linalg.pinv(H)
    dp = inv_H@error_sum    

    ### END CODE HERE ###
    return dp

def subtract_dominant_motion(img1, img2):
    Gx = cv2.Sobel(I, cv2.CV_64F, 1, 0, ksize = 5) # do not modify this
    Gy = cv2.Sobel(I, cv2.CV_64F, 0, 1, ksize = 5) # do not modify this
    th_hi = 0.3 * 256 # you can modify this
    th_lo = 0.15 * 256 # you can modify this

    ### START CODE HERE ###
    # [Caution] From now on, you can only use numpy and 
    # RectBivariateSpline. Never use OpenCV.
    global P
    
    temp = img1.copy()
    img  = img2.copy()
    pd = lucas_kanade_affine(temp, img, P, Gx, Gy)
    P = P + pd
    height, width = img.shape    
    x = np.arange(0,width,1)
    y = np.arange(0,height,1)
    temp_norm = temp/255.0
    temp_spline = RectBivariateSpline(y, x, temp_norm)
    
    temp_warped = img.copy()
    for y_ in y:
        for x_ in x:
            w = np.array([[1+P[0,0], P[2,0]],
                          [P[1,0], 1+P[3,0]]])
            b = np.array([[x_-P[4,0]],[y_-P[5,0]]])
            wx, wy = np.linalg.solve(w,b)
            temp_warped[y_][x_] = float(temp_spline.ev(wy,wx)*255)
    
    moving_image = np.abs(img - temp_warped)
    ### END CODE HERE ###

    hyst = apply_hysteresis_threshold(moving_image, th_lo, th_hi)
    return hyst

if __name__ == "__main__":
    start = time.time()
    data_dir = 'data'
    video_path = 'motion.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_path, fourcc, 150/20, (636, 318))
    tmp_path = os.path.join(data_dir, "organized-{}.jpg".format(0))
    T = cv2.cvtColor(cv2.imread(tmp_path), cv2.COLOR_BGR2GRAY)
    P = np.zeros((6,1))
    for i in range(0, 50):
        img_path = os.path.join(data_dir, "organized-{}.jpg".format(i))
        I = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY)
        clone = I.copy()
        moving_img = subtract_dominant_motion(T, I)
        clone = cv2.cvtColor(clone, cv2.COLOR_GRAY2BGR)
        clone[moving_img, 2] = 522
        out.write(clone)
        T = I
    out.release()
    print((time.time()-start)/60)
    